## Import Data

In [8]:
import pandas as pd

# import moea.csv file from moea.py script
moea = pd.read_csv('output/moea.csv')
# convert time to datetime 
moea['time'] = pd.to_datetime(moea['time'], yearfirst = True).dt.date

# import epa.csv file from epa.py script
epa = pd.read_csv('output/epa.csv')
# convert time to datetime 
epa['time'] = pd.to_datetime(epa['time'], yearfirst = True).dt.date
# replace simplied chinese in city name with traditional chinese
epa['city'] = epa['city'].str.replace('台', '臺')

# import subsidy_amt.csv file from output folder
sub = pd.read_csv('output/subsidy_amt.csv')
# convert time to datetime 
sub['time'] = pd.to_datetime(sub['time'], yearfirst = True).dt.date

## Organise MOEA
EPA and local subsidy policy both had three types of rebatement: purchase of electric scooter (`pur`), elimination of aged gasoline scooter **and** purchase of electric scooter (`eli_pur`), and elimination of aged gasoline scooter (`eli`); however, MOEA only gives subsidy for new purchase (regardless of whether the consumer recycles a gasoline one).

For my research, I intend to look at individual categories, therefore, I have to approximate the number for each type of subsidy for MOEA application data. 

### September 2012 to July 2015

Only MOEA provided subsidy. That is, EPA and local government did not implement subsidy programs at the time. For this period, the number of application for `eli_pur` is half of total application, rounded to integer. As for `pur`, total minus eli_pur. The numbers for `eli` are 0 since no subsidy from MOEA is available for the category.

In [9]:
# pur
moea['新購'] = (moea['moea_app']*0.5).round().astype('int64')
# eli_pur
moea['汰舊換新'] = moea['moea_app'] - moea['新購']
# eli
moea['汰舊'] = 0

# pivot or transpose the data from wide form to long form
moea_long = pd.melt(moea, 
                id_vars=['time', 'city'], 
                value_vars = ['moea_app','新購','汰舊換新','汰舊'], 
                var_name = 'type', 
                value_name = 'moea_app')

# merge with epa on time, city, and type
df = pd.merge(left = epa, 
              right = moea_long[moea_long['type']!='moea_app'], 
              on = ['time','city', 'type'], 
              how = 'outer', 
              indicator = True)

# slice data
before = df[df['_merge']=='right_only'].drop(columns=['_merge']).merge(sub[['time','city','moea_amt']], on=['time', 'city'])
before.head(5)

,time,type,city,epa_app,moea_app,moea_amt
0,2012-09-01,新購,臺北市,NaN,18.0,10000
1,2012-09-01,汰舊換新,臺北市,NaN,19.0,10000
2,2012-09-01,汰舊,臺北市,NaN,0.0,10000
3,2012-09-01,新購,臺中市,NaN,45.0,10000
4,2012-09-01,汰舊換新,臺中市,NaN,45.0,10000


### After August 2015
I first calculated the ratio of `eli_pur` and `pur` from EPA then use the number to approximate the application number of application for MOEA. For example, EPA has 30 `eli_pur`, 70 `pur`, and 12 `eli` applications this month and MOEA has a total of 180 cases for new purchases (regardless of recycle old scooters). The allocation of `eli_pur` is `30/(30+70) = 30%` and `pur` is `70%`. Because MOEA does not subsidise `eli`, the category is not included.

1. Create `after`: entries from `df` that are after August 2015. Drop merging indicator and `moea_app` for now.
2. Create `epa_ind`: `after` grouped by `time`, `city`, and `type` then count of application count for `eli_pur` and `pur`. 
3. Create `epa_r`: divide each category (`epa_ind`) by sum (`epa_ind.groupby(['time', 'city']).transform('sum')`). Reset index to be able to read groupby object and rename column name of `epa_app` to `epa_ratio`. 
4. Create `epa_m`: merge `epa_ind` and `epa_r` on `time`, `city`, and `type` because `eli` was eliminated in step 2 and I need it back.


In [10]:
# Create after for entries from df that are after August 2015. Drop merging indicator and moea_app for now
after = df[df['_merge']!='right_only'].drop(columns = ['moea_app','_merge'])

# Grouped after by time, city, and type with the count of application count for eli_pur and pur
epa_ind = after[after['type'] != '汰舊'].groupby(by = ['time', 'city', 'type'])['epa_app'].sum()

# Divide each category (epa_ind) by sum, reset index to be able to read groupby object and rename column name of epa_app to epa_ratio
epa_r = epa_ind.div(epa_ind.groupby(['time', 'city']).transform('sum')).reset_index().rename(columns = {'epa_app':'epa_ratio'})

# merge epa_ind and epa_r on time, city, and type to get both count and ratio
epa_m = pd.merge(left = epa_ind, right=epa_r, on=['time','city','type'], how='outer')
#epa_m['epa_ratio'] = epa_m['epa_ratio'].fillna(0)

# merge to obtain moea application count
after = epa_m.merge(moea[['city','time','moea_app']], how='left', on=['time','city'], indicator=True)

# multiply ratio with application count
after['moea_app_alloc'] = (after['epa_ratio'] * after['moea_app']).round()
after.loc[(after['moea_app_alloc'] > 0), 'moea_app'] = after['moea_app_alloc']
after = after.drop(columns=['_merge','epa_ratio','moea_app_alloc']).merge(sub, on=['time','city','type'])
after.head(5)

,time,city,type,epa_app,moea_app,epa_amt,local_amt,moea_amt
0,2015-08-01,南投縣,新購,0.0,2.0,5000.0,1200.0,10000
1,2015-08-01,南投縣,汰舊換新,0.0,2.0,8000.0,2100.0,10000
2,2015-08-01,嘉義市,新購,2.0,7.0,5000.0,5000.0,10000
3,2015-08-01,嘉義市,汰舊換新,0.0,7.0,8000.0,6000.0,10000
4,2015-08-01,嘉義縣,新購,0.0,2.0,5000.0,0.0,10000


### Combine Before & After

In [11]:
data = before.append(after).reset_index().drop(columns='index') 

,time,type,city,epa_app,moea_app,moea_amt,epa_amt,local_amt
0,2012-09-01,新購,臺北市,NaN,18.0,10000,NaN,NaN
1,2012-09-01,汰舊換新,臺北市,NaN,19.0,10000,NaN,NaN
2,2012-09-01,汰舊,臺北市,NaN,0.0,10000,NaN,NaN
3,2012-09-01,新購,臺中市,NaN,45.0,10000,NaN,NaN
4,2012-09-01,汰舊換新,臺中市,NaN,45.0,10000,NaN,NaN


## Translate Data
`type` and `city` consist of information in Mandarin. As some libraries do not recognise or display the characters, I am translating these two columns into English. For `city`, use the `city_name.txt` collected earlier from wikipedia. As for `type`, manually input the translation.

In [17]:
city_name = {}

with open('output/city_name.txt', 'r') as f:
    for line in f:
        line = line.strip().split(',')
        city_name.update({line[0]:line[1]})

data = data.replace(city_name).replace({'新購':'New Purchase', '汰舊':'Elimination', '汰舊換新':'Elimination and Purchase'})

In [18]:
data.head(5)

,time,type,city,epa_app,moea_app,moea_amt,epa_amt,local_amt
0,2012-09-01,New Purchase,Taipei City,NaN,18.0,10000,NaN,NaN
1,2012-09-01,Elimination and Purchase,Taipei City,NaN,19.0,10000,NaN,NaN
2,2012-09-01,Elimination,Taipei City,NaN,0.0,10000,NaN,NaN
3,2012-09-01,New Purchase,Taichung City,NaN,45.0,10000,NaN,NaN
4,2012-09-01,Elimination and Purchase,Taichung City,NaN,45.0,10000,NaN,NaN
